In [31]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

def summarise_test(test):
    x ='For {}, in model {}\nAverage NO SIGFALL accuracy is {}\nAverage SIGFALL accuracy is {}\n---\n'
    print(x.format(test['data_name'],test['model'],test['specificity'],test['sensitivity']))

def balance_data_set(dataset, classifier):
    sigfall_indexes = dataset.index[dataset[classifier] == 0].to_list()
    n_falls = len(dataset) - len(sigfall_indexes)
    drop = len(sigfall_indexes) - n_falls
    random.shuffle(sigfall_indexes)
    drop_indexes = sigfall_indexes[0:drop]
    return dataset.drop(drop_indexes)

In [20]:
# Load data sets
inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')
inc_updrs_sigfall_b = balance_data_set(inc_updrs_sigfall, 'SIGFALL')
inc_updrs_sigfall_raw = pd.read_csv('./working_data/normalised_increase_updrs_sigfall_raw.csv')

datasets = [{'data':inc_updrs_sigfall, 'name':'inc_updrs_sigfall'}, 
            {'data':inc_updrs_sigfall_b, 'name':'inc_updrs_sigfall_b'}, 
            {'data':inc_updrs_sigfall_raw, 'name':'inc_updrs_sigfall_raw'}]

In [32]:
models = [QuadraticDiscriminantAnalysis(store_covariance=True)]

tests = []
for m in models:
    for d in datasets:
        tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model QuadraticDiscriminantAnalysis(store_covariance=True)
Average NO SIGFALL accuracy is 0.8499884722933039
Average SIGFALL accuracy is 0.3925613621800571
---

For inc_updrs_sigfall_b, in model QuadraticDiscriminantAnalysis(store_covariance=True)
Average NO SIGFALL accuracy is 0.6724109096717248
Average SIGFALL accuracy is 0.6311259438789543
---

For inc_updrs_sigfall_raw, in model QuadraticDiscriminantAnalysis(store_covariance=True)
Average NO SIGFALL accuracy is 0.8565858336741033
Average SIGFALL accuracy is 0.3762746248325992
---

